In [47]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import re

In [48]:
#Function to load data

# Fonction pour extraire le nom de l'expérience à partir du nom de fichier
def extract_experiment_name(file_name, pattern):
    # Expression régulière pour rechercher le motif "P-V 1V_1#1" dans le nom de fichier
    match = re.search(pattern, file_name)
    if match:
        return match.group()
    else:
        return None

def load_data(data_folder, chip_name, placement, size, graph_type):
    # Liste des dossiers dans le dossier data_folder
    date_folders = os.listdir(data_folder)
    
    # Parcours des dossiers de date
    for date_folder in date_folders:
        # Chemin complet du dossier de date
        date_folder_path = os.path.join(data_folder, date_folder)
        
        # # Liste des dossiers de chips dans le dossier de date
        # chip_folders = os.listdir(date_folder_path)

        # Liste des dossiers de chips dans le dossier de date
        chip_folders = [folder for folder in os.listdir(date_folder_path) if os.path.isdir(os.path.join(date_folder_path, folder))]
        
        
        # Parcours des dossiers de chips
        for chip_folder in chip_folders:
            # Vérification si le nom de la chip correspond
            if chip_name in chip_folder:
                # Chemin complet du dossier de la chip
                chip_folder_path = os.path.join(date_folder_path, chip_folder)
                
                # Liste des fichiers xls dans le dossier de la chip
                xls_files = [file for file in os.listdir(chip_folder_path) if file.endswith('.xls')]
                
                # Parcours des fichiers xls
                for xls_file in xls_files:
                    # Séparation du nom du fichier
                    file_parts = xls_file.split('_')
                    file_placement, file_size, end = file_parts[0], file_parts[1], '_'.join(file_parts[4:])
                    file_graph_type = extract_experiment_name(xls_file, graph_type)
                    #file_graph_type = file_graph_type.split('.')[0]
                    
                    # Vérification si les critères de recherche correspondent
                    if file_placement == placement and file_size == size and file_graph_type == graph_type:
                        # Chargement des données xls dans une DataFrame
                        xls_path = os.path.join(chip_folder_path, xls_file)
                        data = pd.read_excel(xls_path)
                        return data

In [56]:
class Chip:
    def __init__(self, name, file, experience, size, tool=None, temp=None, time=None):
        self.name = name
        self.file = file
        self.experience = experience
        self.size = size
        self.tool = tool
        self.temp = temp
        self.time = time

# Listes de toutes les puces
chips_all = ["3dec02", "3dec08", "3dec09", "3dec10", "3dec11", "3dec14", "3dec15", "3dec16", "3dec17", "3dec18"]

chips_DP = ["3dec17", "3dec11", "3dec09", "dec15"]
chips_SP = ["3dec16", "3dec08", "3dec10", "3dec02", "3dec14"]
chips_600 = ["3dec18"]
chips_500 = ["3dec16", "3dec08", "3dec10", "3dec02", "3dec17", "3dec11"]
chips_450 = ["3dec14", "3dec15", "3dec09"]
chips_20 = ["3dec17", "3dec15", "3dec16", "3dec14"]
chips_120 = ["3dec11", "3dec09", "3dec08", "3dec10"]
chips_240 = ["3dec02"]

list_exp = ["P-V 1V_1#1", "P-V 1V_2#1", "P-V 2V_1#1", "P-V 2V_2#1", "P-V 3V_1#1", "P-V 3V_2#1", 
                      "P-V 4V_1#1", "P-V 4V_2#1", "P-V 3V neg_1#1", "P-V 3V neg_1#1", "PUND 5V_1#1", "PUND 5V neg_1#1", "CV 3V_1#1", "IV 3V_1#1"]

size = ["50", "100", "150", "200", "500"]

def found_tool (chip):
    if chip in chips_DP:
        return "DP"
    if chip in chips_SP:
        return "SP"
    return "WRONG"

def found_temp (chip):
    if chip in chips_600:
        return "600"
    if chip in chips_500:
        return "500"
    if chip in chips_450:
        return "450"
    return "WRONG"

def found_time (chip):
    if chip in chips_20:
        return "20"
    if chip in chips_120:
        return "120"
    if chip in chips_240:
        return "240"
    return "WRONG"


In [57]:
data_folder = "C:\Documents\EPFL\MA4\Projet_de_semestre\Code\Data2"

chip_tri = []
placement = "tr"

for ch in chips_all:
    for exp in list_exp:
        for s in size:
            link = load_data(data_folder, ch, placement, s, exp)
            tool = found_tool(ch)
            temp = found_temp(ch)
            time = found_time(ch)
            chip_treated = Chip(name=ch, file = link, experience = exp, size = s, tool = tool, temp = temp, time = time)
            chip_tri.append(chip_treated)
           


WARNING *** file size (195732) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
     doubleSweepSeg    Vforce         Imeas        Charge          Time  \
0               1.0 -0.003412  3.000397e-08  0.000000e+00  0.000000e+00   
1               NaN -0.000740  3.153789e-07  3.942237e-14  1.250000e-07   
2               NaN -0.000815 -4.976710e-07 -2.278651e-14  2.500000e-07   
3               NaN -0.000739 -1.736594e-07 -4.449394e-14  3.750000e-07   
4               NaN -0.001836  6.725990e-07  3.958093e-14  5.000000e-07   
..              ...       ...           ...           ...           ...   
995             NaN -0.000223 -1.030827e-06  5.823460e-13  1.243750e-04   
996             NaN -0.000660  1.447600e-06  7.632960e-13  1.245000e-04   
997             NaN -0.001516 -1.244812e-06  6.076946e-13  1.246250e-04   
998             NaN  0.001216  7.933345e-07  7.068614e-13  1.247500e-04   
999             NaN  0.000183 -

In [23]:
def Polarisation(chip):
    data = chip.file 
    area = (chip.size*10**(-6))**2
    pol_max = max(data['Charge'])/area
    pol_min = min(data['Charge'])/area
    diff = pol_max - pol_min

    pol_max = pol_max - diff/2
    pol_min = pol_min - diff/2
    return pol_max, pol_min

def Leakage_current(chip):
    #Pas toujours maximum du courant!!! max du voltage et prendre indice pour trouver le courant
    data = chip.file 
    leak_max = max(data['AI'])
    leak_min = min(data['AI'])
    return leak_max, leak_min



In [42]:
inter_list = [chip for chip in chip_tri if chip.temp == "500" and chip.time == "120" and chip.tool == "DP" and chip.experience == "IV 3V_1#1"]
data = inter_list[1].file
print(data)

test = load_data(data_folder, "3dec11", "tr", "200", "IV 3V_1#1")

print(test)


None
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
          Time            BI  BV            AI   AV
0     0.429243 -1.175893e-12   0  4.221565e-13  0.0
1     1.043363  1.207772e-11   0 -6.856072e-12 -0.1
2     1.656534  2.640073e-11   0 -2.121525e-11 -0.2
3     2.269710  4.178934e-11   0 -3.645942e-11 -0.3
4     2.882880  5.856041e-11   0 -5.331109e-11 -0.4
..         ...           ...  ..           ...  ...
116  63.468166  3.070993e-11   0 -2.510704e-11  0.4
117  64.081314  3.370162e-11   0 -2.818787e-11  0.3
118  64.694420  3.726366e-11   0 -3.176978e-11  0.2
119  65.307564  4.139444e-11   0 -3.596386e-11  0.1
120  65.920753  4.610162e-11   0 -4.077015e-11  0.0

[121 rows x 5 columns]


In [35]:
temperature = ["600", "500", "450"]
tool = ["DP", "SP"]
time = ["20", "120", "240"]

class Result:
    def __init__(self, res, tool=None, temp=None, time=None):
        self.res = res
        self.tool = tool
        self.temp = temp
        self.time = time

result = []

for temp in temperature:
    for to in tool:
        for t in time:
            inter_list = [chip for chip in chip_tri if chip.temp == temp and chip.time == t and chip.tool == to]
            list_leak = [chip for chip in inter_list if chip.experience == "IV 3V_1#1"]
            list_pol = [chip for chip in inter_list if chip.experience == "P-V 4V_1#1"]
            compute_leakage = 0
            compute_Polarisation = 0
            mean_leakage = 0
            mean_polarisation = 0
            if len(inter_list) > 0:
                for chip in list_leak:
                    compute_leakage += Leakage_current(chip)[0]
                for chip in list_pol:
                    compute_Polarisation += Polarisation(chip)[0]
                mean_leakage = compute_leakage/len(list_leak)
                mean_polarisation = compute_Polarisation/len(list_pol)
                result_inter = Result(res = mean_leakage, tool = to, temp = temp, time = t)




TypeError: 'NoneType' object is not subscriptable